# Neural Network Force-Free magnetic field extrapolation - NF2
<img src="https://github.com/RobertJaro/NF2/blob/main/images/logo.jpg?raw=true" width="150" height="150">

This notebook provides basic NF2 extrapolations of custom magnetograms. Fill the form below to start the extrapolation.

Run all sections below to start your non-linear force-free extrapolation. Extrapolations take about 6 hours to complete in Google Colab. All results are logged to Weights And Biases (https://wandb.ai). If you want to continue an interrupted run, fill the WandB section.

GitHub Page: https://github.com/RobertJaro/NF2


#Install NF2

In [ ]:
!pip install git+https://github.com/RobertJaro/NF2

from nf2.extrapolate import run

# Extrapolation

In [ ]:
#@markdown select directory for saving extrapolation results
base_path = 'extrapolation_result' #@param {type:"string"}

In [ ]:
#@title Data configuration

#@markdown Insert FITS files
br_path = '' #@param {type:"string"}
bt_path = '' #@param {type:"string"}
bp_path = '' #@param {type:"string"}

#@markdown Insert FITS error files (optional)
br_error_path = '' #@param {type:"string"}
bt_error_path = '' #@param {type:"string"}
bp_error_path = '' #@param {type:"string"}

br_error_path = None if br_error_path == '' else br_error_path
bt_error_path = None if bt_error_path == '' else bt_error_path
bp_error_path = None if bp_error_path == '' else bp_error_path

#@markdown Load header information from SunPy Map or only read the FITS data
load_map = False #@param {type:"boolean"}

#@markdown Insert the pixel size in Mm (SHARP = 0.36 Mm)
Mm_per_pixel = 0.36 #@param {type:"number"}

data_config = {
  'type': 'fits',
  'slices': {
      'fits_path' : {
        'Br': br_path,
        'Bt': bt_path,
        'Bp': bp_path
      },
      'Mm_per_pixel': Mm_per_pixel,
      'load_map': load_map,
  },
  'iterations': 10000
}

if br_error_path is not None and bt_error_path is not None and bp_error_path is not None:
  data_config['slices']['error_path'] = {'Br_err': br_error_path, 'Bt_err': bt_error_path, 'Bp_err': bp_error_path}

In [ ]:
#@title Logging Settings
entity = "" #@param {type:"string"}
project = "nf2" #@param {type:"string"}
#@markdown Enter your credentials if you want to continue an interrupted run. This will restore the latest model and continue the training.
continue_training = False #@param {type:"boolean"}
id = "" #@param {type:"string"}

entity = None if entity == "" else entity
project = None if project == "" else project
logging_config = {'log_model': 'all', 'entity': entity, 'project': project}

if continue_training:
  logging_config['id'] = id

In [ ]:
#@title Advanced model settings
use_custom_settings = False #@param {type:"boolean"}
#@markdown Set `vector_potential` if you want to run a divergence-free extrapolation
vector_potential = False #@param {type:"boolean"}
#@markdown Change `dim` to set the available free parameters. Lower values reduce the runtime but may decrease the quality of the extrapolation. Use 256 for the standard setting and 64 for quick tests.
dim = 256 #@param {type:"number"}

if use_custom_settings:
  model_config = {'type': 'vector_potential' if vector_potential else 'b',
                  'dim': dim}
else:
  model_config = None

In [ ]:
#@title Advanced training settings
use_custom_settings = False #@param {type:"boolean"}
#@markdown Change the weighting of the force-free condition
lambda_ff = 0.1 #@param {type:"number"}
#@markdown Change the weighting of the divergence-free condition
lambda_div = 0.1 #@param {type:"number"}
#@markdown Change the weighting of the boundary condition
lambda_boundary = 1.0 #@param {type:"number"}



if use_custom_settings:
  boundary_loss = {'type': 'boundary', 'name': 'boundary', 'lambda': lambda_boundary,
                 'ds_id': ['boundary_01', 'potential']}
  force_free_loss = {'type': 'force_free', 'lambda': lambda_ff}
  divergence_free_loss = {'type': 'divergence', 'lambda': lambda_div}

  training_config = {'loss_config': [boundary_loss, force_free_loss, divergence_free_loss]}
else:
  training_config = {}

## Start Extrapolation
For the first run you will be redirected to the WandB login. Login with your existing user or register a new user to log your results online.

In [ ]:
run(base_path, data=data_config,
    logging=logging_config,
    training=training_config,
    model=model_config)